# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

## Reading the data from csv file

In [263]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [264]:
# Loading data from csv file
import pandas as pd
data_path = '/gdrive/MyDrive/datasets/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [265]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [266]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [267]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [268]:
# print(train_data['category'].value_counts())
# train_data=train_data.drop_duplicates(['text'])
# print(train_data['category'].value_counts())

In [269]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

In [275]:
# Imports
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [276]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

# STEP - 16: The cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
# STEP - 17:Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.
def Cosine_Similarity(x, y):
      # Cos_Sim(x, y) = x . y / ||x|| * ||y||
      if (np.linalg.norm(x)==0 or np.linalg.norm(y)==0):
        # WARNING HANDLED: Not going to happen until we have zero vector - Which in our case won't happended as atleast one word will present in the document.
        return 1
      return float(np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))
      
def Majority_Wins(preds_labels):
      sorted(preds_labels)
      return max(set(preds_labels), key = preds_labels.count)

def hash_function(query_vec,planes_arr):
      hash_str=""
      for plane_vec in planes_arr:
            if np.dot(query_vec,plane_vec)>0:
                hash_str+='1'
            else:
                hash_str+='0'
      # print(hash_str)
      return hash_str


def predictLabels (test_data):
      """
      Given the test_data, return the labels for all the rows in the test data.
      Follow the step by step instructions mentioned above.
      """

      np.random.seed(1)

      ##############################################################
      ####  Write YOUR CODE BELOW  as per the above instructions ###
      ##############################################################
      """
      Given the train data, it will create buckets of similar documents and returns the buckets and the planes that we used.
      Why planes? -> It will use to create same hashes and classify.
      """
      final_label_prediction=[]
      # STEP - 2: Vectorize train_data using sklearns built in tfidf vectorizer.
      # STEP - 3: Ignore unigrams and make use of both bigrams & trigrams and also limit the max features to 4000 and minimum document frequency to 10.
      # Bigram & Trigrams [Default unigram only (1,1)]
      n_grams = (2,3)
      # Max Features
      n_features = 4000
      # min doc frequency [Default=1]
      min_doc_freq = 10
      
      # This is was critical. Removing Stopwords were giving worst accuracy due to bi-gram & Tri-gram usage. limiting this works. while keeping the Stopwords.
      max_doc_freq = 0.75 # (If a word is appearing for more than 75% of data than it is not much important to distinguish the class)
      
      # instantiate the vectorizer object
      vectorizer = TfidfVectorizer(ngram_range=n_grams, min_df=min_doc_freq, max_df=max_doc_freq, max_features=n_features)
      # convert the text input column into a matrix with tfidf vectorizor
      X_train_transformed = vectorizer.fit_transform(train_data['text'])
      # print("Train data shape:")
      # print(X_train_transformed.shape)


      # STEP - 4: After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
      n_planes=5
      dims=X_train_transformed.shape[1]
      plane_norms=[]
      for i in range(n_planes):
              plane_norms.append(np.random.normal(size=(dims)))

      # STEP - 5: As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
      X_train_narr=X_train_transformed.todense()            # Converting scipy sparse mat to Numpy array
      buckets={}
      # Generating hash and adding them to a buckets dict.    => buckets{<hash>}=[<doc_id>,...,<doc_id>]
      for i_doc in range(len(X_train_narr)):
              hashkey=hash_function(X_train_narr[i_doc].tolist(),plane_norms)
              # Check if the generated hash is present then add the document in the bucket.
              if hashkey not in buckets:
                    buckets[hashkey]=[]
              buckets[hashkey].append(i_doc)
      # print(buckets.keys())

      # STEP - 8:  Once the hash table is generated now take in each of the query points from the test data.
      # STEP - 9:  Vectorize those query points using the same tfidf vectorizer as mentioned above.
      # print("Test data shape:")
      X_test_transformed = vectorizer.transform(test_data['text'])
      # print(X_test_transformed.shape)
      X_test_narr=X_test_transformed.toarray() 
      
      tests=1
      # STEP - 10: Now use the hash function on this query point and fetch all the similar data points from the hashtable.
      for test_doc in X_test_narr:
          hashkey=hash_function(X_train_narr[i_doc].tolist(),plane_norms)
                     
          nbor_list=buckets[hashkey]
                  
          # STEP - 11: Use cosine similarity to compute 11-Nearest Neighbours from the list of data points obtained in the above step.
          # All Neighbours similarity calculation
          similarity_list=[]
          for nbor in nbor_list:               
                similarity_list.append(Cosine_Similarity(X_train_narr[nbor].tolist(),test_doc.tolist()))

          nbor_list=np.array(nbor_list)
          # Selecting top 11
          K_neighbors=11
          sorted_idx_arr = np.argsort(similarity_list)
          nearest_K=nbor_list[sorted_idx_arr][-K_neighbors:]
          # print(nearest_K)

          # Step - 12 : Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
          k_predictions=[train_data['category'][i] for i in nearest_K]
          print("For test{0} -> {1} ".format(tests, sorted(k_predictions)))
          tests+=1
          

          # Step - 13 : In case of a tie in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels alphabetically(A-Z), 
          # i.e. for example labels starting with A would get more preference than labels starting with Z.
          final_label_prediction.append(Majority_Wins(k_predictions))
      
      # STEP - 14 & 15: Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
      return final_label_prediction


predictLabels(test_data)

For test1 -> ['business', 'business', 'business', 'business', 'entertainment', 'sport', 'tech', 'tech', 'tech', 'tech', 'tech'] 
For test2 -> ['business', 'business', 'business', 'entertainment', 'entertainment', 'politics', 'sport', 'sport', 'tech', 'tech', 'tech'] 
For test3 -> ['business', 'politics', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech'] 
For test4 -> ['entertainment', 'politics', 'politics', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport'] 
For test5 -> ['business', 'business', 'business', 'business', 'entertainment', 'sport', 'sport', 'tech', 'tech', 'tech', 'tech'] 
For test6 -> ['business', 'business', 'business', 'business', 'business', 'business', 'business', 'sport', 'sport', 'sport', 'sport'] 
For test7 -> ['business', 'business', 'entertainment', 'entertainment', 'entertainment', 'politics', 'politics', 'politics', 'politics', 'tech', 'tech'] 
For test8 -> ['entertainment', 'entertainment', 'entertainment', 'entertai

['tech',
 'business',
 'tech',
 'sport',
 'business',
 'business',
 'politics',
 'entertainment',
 'tech',
 'sport']

## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [277]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


For test1 -> ['business', 'business', 'business', 'business', 'entertainment', 'sport', 'tech', 'tech', 'tech', 'tech', 'tech'] 
For test2 -> ['business', 'business', 'business', 'entertainment', 'entertainment', 'politics', 'sport', 'sport', 'tech', 'tech', 'tech'] 
For test3 -> ['business', 'politics', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech', 'tech'] 
For test4 -> ['entertainment', 'politics', 'politics', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport'] 
For test5 -> ['business', 'business', 'business', 'business', 'entertainment', 'sport', 'sport', 'tech', 'tech', 'tech', 'tech'] 
For test6 -> ['business', 'business', 'business', 'business', 'business', 'business', 'business', 'sport', 'sport', 'sport', 'sport'] 
For test7 -> ['business', 'business', 'entertainment', 'entertainment', 'entertainment', 'politics', 'politics', 'politics', 'politics', 'tech', 'tech'] 
For test8 -> ['entertainment', 'entertainment', 'entertainment', 'entertai